## Ensemble learning
Ensemble vem de conjunto, ou seja, um grupo de preditores e chamado de ensemble<br>
um exemplo seria fazer arvores de decisoes separadas para partes aleatorias do <br>
dataset e decidirem qual e a resposta. A maioria vence ou sabedoria das multidoes

## Voting Classifiers

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
iris = datasets.load_iris()
X = iris['data'][:, (2, 3)]
y = iris['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [3]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
                estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
                voting = 'hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [4]:
from sklearn.metrics import accuracy_score

In [5]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 1.0
RandomForestClassifier 1.0
SVC 1.0
VotingClassifier 1.0


#### MEGA OVERFIT

## Baggin e Pasting

### Baggin
Eh quando usamos o mesmo algoritmo para varios subsets. Quando o sampling e performado com reposicao e chamado de baggin <br>
quando nao tem reposicao se chama pasting

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [7]:
bag_clf = BaggingClassifier(
            DecisionTreeClassifier(), n_estimators = 500,
            max_samples = 100, bootstrap = True, n_jobs = -1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

## Out-of-bag Evaluation

In [8]:
bag_clf = BaggingClassifier(
        DecisionTreeClassifier(), n_estimators = 500,
        bootstrap = True, n_jobs = -1, oob_score = True)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9428571428571428

In [9]:
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

## Random Patches e Random Subspaces

É quando ocorre sampling das features também. Sampling é controlado por dois hyperparametros: max_features e bootstrap_features.<br>
Funcionam da mema maneira que max_samples e bootstrap. Isso é util quando está lidando com inputs de alta dimensionalidade.<br>
Mantendo todas as instancias de treino bootstrap = False, max_samples = 1. Mas para sampling features bootstrap = True ou max_features para um valor menor que 1

## Random Forests

Random forest é um metodo de ensemble em um grupo de decision trees, geralmente treinado por bagging ou as vezes pasting, <br>
tipicamente com max_samples setado para o tamanho do set de treinamento ao enves de construir um baggingclassifier e passar decisiontreeclassifier <br>
pode utilizar RandomForestClassifier que é mais conveniente e tomizado para decision trees. O código abaixo usa toda a CPU com 500 claissificadores e com 16 nós

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
rnd_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

Random forest usado da maneira acima generaliza melhor e perde o bias

In [12]:
bag_clf = BaggingClassifier(
            DecisionTreeClassifier(splitter = 'random', max_leaf_nodes = 16),
            n_estimators = 500, max_samples = 1, bootstrap = True, n_jobs = -1)


É possivel fazer ainda mais aleatorios que a Random Forest

In [13]:
from sklearn.ensemble import ExtraTreesClassifier

In [14]:
extra_clf = ExtraTreesClassifier(max_leaf_nodes = 16)
extra_clf.fit(X_train, y_train)

ExtraTreesClassifier(max_leaf_nodes=16)

## Feature importance

In [15]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators = 500, n_jobs = -1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.10983757873191423
sepal width (cm) 0.023445688608524216
petal length (cm) 0.4482697372991856
petal width (cm) 0.4184469953603759


## Boosting

É quando vários weak learners juntos em um ensemble formam
um strong learner <br>

In [16]:
from sklearn.ensemble import AdaBoostClassifier

In [17]:
ada_clf = AdaBoostClassifier(
            DecisionTreeClassifier(max_depth = 1), n_estimators = 200,
            algorithm = 'SAMME.R', learning_rate = 0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

Em caso de overfit diminua o numero de estimadores ou regularize mais os estimadores base

## Gradient Boosting

Outro algoritmo de boosting famoso é o gradiente boosting. Assim como adaboost, o gradient boosting funciona adicionando preditores a uma ensemble cada um corrigindo seu antecessor. <br>
Diferentemente do AdaBoost ele não altera os pesos por iteração e sim ajusta o predito a partir do erro residual feito pelo seu antecessor.

In [18]:
from sklearn.tree import DecisionTreeRegressor

In [19]:
tree_reg1 = DecisionTreeRegressor(max_depth = 2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [20]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth = 2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

In [21]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth = 2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [22]:
y_pred = sum(tree.predict(X) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [23]:
from sklearn.ensemble import GradientBoostingRegressor

In [24]:
gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 3, learning_rate = 1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

O hyperparametro learning_rate escala a contribuicao de cada arvore. Se setar para valores baixos como 0.1 <br>
sera necesserario mais arvores para ensemble fittar o training set mas geralmente generalizara melhor. isso <br>
e uma regularizacao chamada shrinkage. Para achar o numero otimo de numero de arvores pode usar early stopping <br>
uma maneira simples e usando o metodo staged_predict().

In [25]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
best_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth = 2, n_estimators = best_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=97)

Fazendo early stopping na mao seria da seguinte maneira

In [27]:
gbrt = GradientBoostingRegressor(max_depth = 2, warm_start = True)

min_val_error = float('inf')
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break

In [28]:
import xgboost

In [29]:
xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [30]:
xgb_reg.fit(X_train, y_train, eval_set = [(X_val, y_val)], early_stopping_rounds = 2)
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.68782
[1]	validation_0-rmse:0.49611
[2]	validation_0-rmse:0.36554
[3]	validation_0-rmse:0.28924
[4]	validation_0-rmse:0.23379
[5]	validation_0-rmse:0.21105
[6]	validation_0-rmse:0.19163
[7]	validation_0-rmse:0.18134
[8]	validation_0-rmse:0.17584
[9]	validation_0-rmse:0.17736
[10]	validation_0-rmse:0.17914


/home/matheus/.local/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


## Stacking